#### Objetive of the Task reduce the Trainable param 8,000 & accuracy is above 99.4%"

In [1]:
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import np_utils

In [2]:
mnist=tf.keras.datasets.mnist

In [3]:
(x_train_full,y_train_full),(x_test,y_test)=mnist.load_data()

In [4]:
(x_train,x_valid)=x_train_full[5000:],x_train_full[:5000]
(y_train,y_valid)=y_train_full[5000:],y_train_full[:5000]

In [5]:
x_train.shape

(55000, 28, 28)

In [6]:
num_classes=len(np.unique(y_train))


y_train=np_utils.to_categorical(y_train,num_classes)

y_test=np_utils.to_categorical(y_test,num_classes)
y_valid=np_utils.to_categorical(y_valid,num_classes)

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Activation

from keras.layers import BatchNormalization,Dropout, LeakyReLU
from tensorflow.keras.callbacks import Callback, EarlyStopping, ReduceLROnPlateau

In [8]:
# CONV-BATCH-ACTIVATION_POOLING
model=Sequential()

model.add(Conv2D(filters=32,kernel_size=3,input_shape=(28,28,1))) #input - Kernal+1 =>26x26x8
model.add(BatchNormalization())
model.add(LeakyReLU())

model.add(MaxPooling2D(pool_size=2)) #12x12x16
model.add(Conv2D(filters=14,kernel_size=3)) #input - Kernal+1 =>26x26x8
model.add(BatchNormalization())
model.add(Activation('LeakyReLU'))
model.add(MaxPooling2D(pool_size=2)) #5x5x16

model.add(Conv2D(filters=10,kernel_size=5)) #5x5x10
model.add(Flatten())

model.add(Activation('softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 26, 26, 32)       128       
 ormalization)                                                   
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 26, 26, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 14)        4046      
                                                                 
 batch_normalization_1 (Batc  (None, 11, 11, 14)       5

In [9]:
early_stopping = EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    min_delta=0.005,
    baseline=0.98,
    patience=10,
    restore_best_weights=True,
    verbose=1
)

lr_scheduler = ReduceLROnPlateau(
    monitor='val_accuracy',
    patience=5,
    factor=0.2,
    min_lr=1e-6,
    verbose=1
)

In [10]:
callbacks = [
    #early_stopping,
    lr_scheduler,
]

In [11]:
#Here weight /Kernal will update
from tensorflow.keras import optimizers

loss = 'categorical_crossentropy'
optim = optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, name='Nadam')
batch_size = 128


In [12]:
  model.compile(
        loss=loss,
        optimizer=optim,
        metrics=['accuracy']
    )

In [13]:
#from keras.callbacks import ModelCheckpoint,EarlyStopping,LearningRateScheduler
epochs=50
VALIDATION_DATA=(x_valid,y_valid)

model.fit(x_train,y_train,batch_size=batch_size,
    epochs=epochs,validation_data=VALIDATION_DATA,callbacks=callbacks,
    use_multiprocessing=True)

Epoch 1/50
430/430 [==============================] - 52s 118ms/step - loss: 0.3058 - accuracy: 0.9173 - val_loss: 0.0975 - val_accuracy: 0.9712 - lr: 0.0010
Epoch 2/50
430/430 [==============================] - 51s 118ms/step - loss: 0.0826 - accuracy: 0.9761 - val_loss: 0.0688 - val_accuracy: 0.9792 - lr: 0.0010
Epoch 3/50
430/430 [==============================] - 51s 119ms/step - loss: 0.0602 - accuracy: 0.9822 - val_loss: 0.0514 - val_accuracy: 0.9834 - lr: 0.0010
Epoch 4/50
430/430 [==============================] - 51s 119ms/step - loss: 0.0498 - accuracy: 0.9850 - val_loss: 0.0461 - val_accuracy: 0.9862 - lr: 0.0010
Epoch 5/50
430/430 [==============================] - 51s 119ms/step - loss: 0.0431 - accuracy: 0.9872 - val_loss: 0.0541 - val_accuracy: 0.9840 - lr: 0.0010
Epoch 6/50
430/430 [==============================] - 51s 118ms/step - loss: 0.0383 - accuracy: 0.9884 - val_loss: 0.0418 - val_accuracy: 0.9870 - lr: 0.0010
Epoch 7/50
430/430 [==============================] 

In [14]:
# evaluate test accuracy
score = model.evaluate(x_test, y_test, verbose=0)
accuracy = 100*score[1]

# print test accuracy
print('Test accuracy: %.4f%%' % accuracy)

Test accuracy: 98.9200%
